In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
data = pd.read_csv('data/BO_3w.csv')

Q_11 = data['Q11']
Q_22 = data['Q22']
Q_33 = data['Q33']

alphas_read = data['alpha']

In [9]:
from numba import njit
from tqdm import tqdm
from scipy.linalg import sqrtm


@njit
def softmax(x, beta=1):
    max_x = np.array([np.max(row) for row in x])

    x = x - max_x.reshape(-1,1)
    P = np.exp(beta*x)    
    return (P.T / np.sum(P,1)).T


@njit
def A(z1):
    return softmax(np.outer(z1,z1)) + np.eye(len(z1))


@njit
def B(z1, z2):
    A1 = A(z1)
    A2 = A(A1 @ z2)
    return A2 @ A1


@njit
def g(z1, z2, z3):
    B1 = B(z1, z2) @ z3
    return softmax(np.outer(B1, B1))


samples = 100000
gen_error_list = np.zeros(len(Q_11))

for i in tqdm(range(len(Q_11))):
    Q = np.array([[Q_11[i], 0, 0], [0, Q_22[i], 0], [0, 0, Q_33[i]]])
    gen_error = 0
    for _ in range(samples):

        Z = np.random.normal(0,1, (3,2))
        U = np.random.normal(0,1, (3,2))

        sqrt_Q = sqrtm(Q)
        sqrt_one_minus_Q = sqrtm(np.eye(Q.shape[0]) - Q)
        omega = sqrt_Q @ Z

        Z_true = sqrt_Q@Z + sqrt_one_minus_Q@U
        y_teacher = g(Z_true[0], Z_true[1], Z_true[2])
        y_student = g(omega[0], omega[1], omega[2])

        loss = np.linalg.norm(y_teacher - y_student)**2
        gen_error += loss

    gen_error_list[i] = gen_error / samples

    print(f"alpha = {alphas_read[i]}: {gen_error_list[i]}")


  0%|          | 0/49 [00:00<?, ?it/s]

  2%|▏         | 1/49 [00:28<22:34, 28.21s/it]

alpha = 0.0: 0.5849756569798746


  4%|▍         | 2/49 [00:50<19:29, 24.89s/it]

alpha = 0.0625: 0.5843271788812764


...

alpha = 2.875: 0.07511244319613518


 98%|█████████▊| 48/49 [18:38<00:22, 22.97s/it]

alpha = 2.9375: 0.074726974485367


100%|██████████| 49/49 [19:01<00:00, 23.29s/it]

alpha = 3.0: 0.07403265889743416


In [10]:
[print(f"{alphas_read[i]},{gen_error_list[i]}") for i in range(len(alphas_read))]

1

0.0,0.5849756569798746
0.0625,0.5843271788812764
0.125,0.5834411704371758
0.1875,0.5841458338923836
0.25,0.5784702115464376
0.3125,0.5874128951216233
0.375,0.5908169279305303
0.4375,0.5549054593102759
0.5,0.4725599407910743
0.5625,0.3770265772449534
0.625,0.3001588075886262
0.6875,0.2404559458460032
0.75,0.2054596889922188
0.8125,0.1840042022135637
0.875,0.1628542563306462
0.9375,0.15204252475218893
1.0,0.14370831268090817
1.0625,0.1361410805308254
1.125,0.12999479499456443
1.1875,0.12128936048124529
1.25,0.11667110363062329
1.3125,0.11161167314817247
1.375,0.1061489926757768
1.4375,0.10057491332648912
1.5,0.0977731757365553
1.5625,0.09551362671395895
1.625,0.09371748702223848
1.6875,0.090564976108122
1.75,0.0891716449742239
1.8125,0.08665048212259818
1.875,0.08643279862713567
1.9375,0.08432369870237369
2.0,0.08330255198251221
2.0625,0.08331761630331404
2.125,0.08226766152639862
2.1875,0.08216093166618896
2.25,0.08055844264457905
2.3125,0.07949326612151697
2.375,0.07809164174904266
2.4

1

In [11]:
from numba import njit
from tqdm import tqdm
from scipy.linalg import sqrtm

@njit
def softmax(x, beta=1):
    max_x = np.array([np.max(row) for row in x])

    x = x - max_x.reshape(-1,1)
    P = np.exp(beta*x)    
    return (P.T / np.sum(P,1)).T


@njit
def A(z1):
    return softmax(np.outer(z1,z1)) + np.eye(len(z1))


@njit
def B(z1, z2):
    A1 = A(z1)
    A2 = A(A1 @ z2)
    return A2 @ A1


@njit
def g(z1, z2, z3):
    B1 = B(z1, z2) @ z3
    return softmax(np.outer(B1, B1))


samples = 100000
gen_error_list = np.zeros(len(Q_11))

for i in tqdm(range(len(Q_11))):
    Q = np.array([[0, 0, 0], [0, 0, 0], [0, 0, Q_33[i]]])
    gen_error = 0
    for _ in range(samples):

        Z = np.random.normal(0,1, (3,2))
        U = np.random.normal(0,1, (3,2))

        sqrt_Q = sqrtm(Q)
        sqrt_one_minus_Q = sqrtm(np.eye(Q.shape[0]) - Q)
        omega = sqrt_Q @ Z

        Z_true = sqrt_Q@Z + sqrt_one_minus_Q@U
        y_teacher = g(Z_true[0], Z_true[1], Z_true[2])
        y_student = g(omega[0], omega[1], omega[2])

        loss = np.linalg.norm(y_teacher - y_student)**2
        gen_error += loss

    gen_error_list[i] = gen_error / samples

    print(f"alpha = {alphas_read[i]}: {gen_error_list[i]}")

  2%|▏         | 1/49 [00:28<23:00, 28.77s/it]

alpha = 0.0: 0.5841452984069229


  4%|▍         | 2/49 [00:52<20:07, 25.70s/it]

alpha = 0.0625: 0.5840106244987391


  6%|▌         | 3/49 [01:14<18:33, 24.20s/it]

...

alpha = 2.875: 0.11146171027407747


 98%|█████████▊| 48/49 [19:01<00:23, 23.64s/it]

alpha = 2.9375: 0.11253263114575122


100%|██████████| 49/49 [19:23<00:00, 23.74s/it]

alpha = 3.0: 0.11247131164153605


In [12]:
[print(f"{alphas_read[i]},{gen_error_list[i]}") for i in range(len(alphas_read))]

1

0.0,0.5841452984069229
0.0625,0.5840106244987391
0.125,0.5833305157070101
0.1875,0.5851799457682222
0.25,0.5794285750184182
0.3125,0.5860315865001978
0.375,0.5854370582668691
0.4375,0.5488822019991927
0.5,0.47451091815640634
0.5625,0.38075774675651775
0.625,0.30137274190244573
0.6875,0.2411445893582055
0.75,0.20552106278741253
0.8125,0.18110879807634797
0.875,0.16365309393769106
0.9375,0.15398749486824034
1.0,0.14440068405413048
1.0625,0.13874995950267413
1.125,0.13389340602128216
1.1875,0.1317563043301489
1.25,0.12906308710810135
1.3125,0.12575084139374487
1.375,0.12323934912571877
1.4375,0.12121165810981263
1.5,0.12074590135326058
1.5625,0.11696317449049205
1.625,0.1174693109362527
1.6875,0.11790824450203112
1.75,0.1164068270837184
1.8125,0.11684210555907215
1.875,0.1156201537241975
1.9375,0.11538462882535137
2.0,0.11638639669674067
2.0625,0.11410085331050623
2.125,0.11496516691860562
2.1875,0.11354679413501043
2.25,0.11500764532288894
2.3125,0.11302030542398575
2.375,0.1118007374792

1